In [10]:
from utils_csa import save_obj, load_obj

import pandas.io.sql as psql
import pandas as pd
import numpy as np

from pytz import timezone
from datetime import datetime, timedelta, date

# load rf classifier already fitted
clf = load_obj('clf_rf_2018_11_19')

# load test data
X_test = load_obj('clf_rf_2018_11_19_X_test')
y_test = load_obj('clf_rf_2018_11_19_y_test')

In [6]:
# inspiration backtrader https://www.backtrader.com/docu/quickstart/quickstart.html
# Goal : Create a class that is generic and can be used easialy

# commissions %
# buy conditions
# sell conditions
# init amount $
# bet size (dépend proba ? tjrs même au début) $
# 